# Calculating Galaxy Radial Velocities

In this portion of the project, we will walk through the steps to use the `astroquery` library to download spectra from the Sloan Digital Sky Survey (SDSS) for a given galaxy. We will identify the most prominent emission line in the spectrum, calculate the redshift, and determine the radial velocity. These calculations are essential for understanding the expansion of the universe using Hubble's law.

## Analyzing SDSS spectra

To begin, let's install the required libraries using pip:

In [23]:
%%capture
!pip install astroquery matplotlib numpy bokeh ipywidgets

### Step 1: Import Libraries

First, let's import the necessary libraries:

In [24]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
import astropy.units as u
import ipywidgets as w

### Step 2: Querying SDSS for Spectra

To download the spectra of a galaxy, we need its coordinates. Let's assume you have the right ascension (RA) and declination (Dec) of the galaxy.

In [25]:
# Coordinates of the galaxy
position = SkyCoord(2.02345833, 14.83980556, unit="deg")

# Query SDSS for spectra
sp_query = SDSS.query_region(position, spectro=True, radius=0.1 * u.arcsec)
sp_id = sp_query[0]['specobjid']
sp_data = SDSS.get_spectra(matches=sp_query)

# Extract the spectrum from the FITS file
hdu_list = sp_data[0]
hdu_list.info()

# The spectrum data is usually in the first HDU
spectrum = hdu_list[1].data
flux = spectrum['flux']
wavelength = 10**spectrum['loglam']  # Convert log wavelength to linear scale

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     139   ()      
  1  COADD         1 BinTableHDU     26   3841R x 8C   [E, E, E, J, J, E, E, E]   
  2  SPECOBJ       1 BinTableHDU    262   1R x 126C   [6A, 4A, 16A, 23A, 16A, 8A, E, E, E, J, E, E, J, B, B, B, B, B, B, J, 22A, 19A, 19A, 22A, 19A, I, 3A, 3A, 1A, J, D, D, D, E, E, 19A, 8A, J, J, J, J, K, K, J, J, J, J, J, J, K, K, K, K, I, J, J, J, J, 5J, D, D, 6A, 21A, E, E, E, J, E, 24A, 10J, J, 10E, E, E, E, E, E, E, J, E, E, E, J, E, 5E, E, 10E, 10E, 10E, 5E, 5E, 5E, 5E, 5E, J, J, E, E, E, E, E, E, 25A, 21A, 10A, E, E, E, E, E, E, E, E, J, E, E, J, 1A, 1A, E, E, J, J, 1A, 5E, 5E]   
  3  SPZLINE       1 BinTableHDU     48   29R x 19C   [J, J, J, 13A, D, E, E, E, E, E, E, E, E, E, E, J, J, E, E]   
  4  B1-00012103-00012101-00012102    1 BinTableHDU    146   2044R x 7C   [E, E, E, J, E, E, E]   
  5  B1-00012104-00012101-00012102    1 BinTa

### Step 3: Identifying the most prominent emission lines

Visualizing the spectrum helps in identifying prominent emission lines. Note here that we use an interactive plotting library calling Plotly to plot the spectrum. This is so we can use the interactive features of Plotly to zoom in and identify the emission lines more accurately. Notice that there are, in fact, many prominent emission lines in our spectrum. In general, lines like H-alpha, H-beta, OIII, and NII are common in galaxy spectra, and in this one, it seems that OIII is the most prominent.

It does take some experience to see a spectrum and know, generally, which lines are which even if they are redshifted (or blueshited). For OIII, we know that its rest wavelength is 5007 Angstroms, so we can use that information to identify it in the spectrum. Let's add a vertical line that represents the rest wavelength of OIII. We will also add verticale lines for H-alpha and H-beta for use in other galaxies where those lines might be most prominent.

In [26]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [27]:
from bokeh.models import HoverTool, Span

fig = figure(title="Galaxy Spectrum", x_axis_label='Wavelength [Angstrom]', y_axis_label='Flux [$10^{-17}$ erg/s/cm^2/Angstrom]', height=400, sizing_mode="stretch_width")

hover = HoverTool()
hover.tooltips = [
    ("X", "@x"),
    ("Y", "@y"),
]

# Add the hover tool to the figure
fig.add_tools(hover)

fig.line(wavelength, flux, line_color='black', legend_label='Spectrum')

# OIII
oiii_rest_wavelength = 5007
fig.add_layout(Span(location=oiii_rest_wavelength, dimension='height', line_color='red', line_dash='dashed', line_width=1))

# H-alpha
halpha_rest_wavelength = 6563
fig.add_layout(Span(location=halpha_rest_wavelength, dimension='height', line_color='green', line_dash='dashed', line_width=1))

# H-beta
hbeta_rest_wavelength = 4861
fig.add_layout(Span(location=hbeta_rest_wavelength, dimension='height', line_color='blue', line_dash='dashed', line_width=1))

show(fig)

Using the figure above, identify the center of the OIII emission feature by zooming in and hovering your cursor over the peak of the emission line. Doing so, we find a peak at around 5236 Angstrom.

### Step 4: Calculating the Redshift

Recall that the redshift ($z$) is calculated using the observed and rest wavelengths:

$$ z = \frac{\lambda_{\text{observed}} - \lambda_{\text{emitted}}}{\lambda_{\text{emitted}}} $$

In [28]:
observed_wavelength = 5236  # Wavelength of the measured emission line
rest_wavelength = 5007  # Rest wavelength of the emission line

redshift = (observed_wavelength - rest_wavelength) / rest_wavelength

print(f"Redshift of the OIII emission line: {redshift:.4f}")

Redshift of the OIII emission line: 0.0457


Let's check our redshift again the other emission lines in the spectrum. We can do this by applying the redshift to the rest wavelengths we defined above. Rearranging our redshift formula, we can solve for the observed wavelength:

$$ \lambda_{\text{observed}} = (1 + z) \times \lambda_{\text{emitted}} $$

In [29]:
halpha_observed_wavelength = (1 + redshift) * halpha_rest_wavelength

print(f"Observed wavelength of H-alpha emission line: {halpha_observed_wavelength:.2f}")

hbeta_observed_wavelength = (1 + redshift) * hbeta_rest_wavelength

print(f"Observed wavelength of H-beta emission line: {hbeta_observed_wavelength:.2f}")

Observed wavelength of H-alpha emission line: 6863.17
Observed wavelength of H-beta emission line: 5083.32


Let's de-redshift the entire spectrum and see if it lines up with the rest wavelengths of our emission lines. We'll over-plot this de-redshifted spectrum on the original to see the full effect of expansion. Rearranging our equation above, we find that

$$ \lambda_{\text{emitted}} = \frac{\lambda_{\text{observed}}}{1 + z} $$

In [30]:
# De-redshift the spectrum
de_redshifted_wavelength = wavelength / (1 + redshift)

# Create a plot with the original and de-redshifted spectrum
fig = figure(title="Galaxy Spectrum", x_axis_label='Wavelength [Angstrom]', y_axis_label='Flux [$10^{-17}$ erg/s/cm^2/Angstrom]', height=400, sizing_mode="stretch_width")

hover = HoverTool()
hover.tooltips = [
    ("X", "@x"),
    ("Y", "@y"),
]

# Add the hover tool to the figure
fig.add_tools(hover)

# Original spectrum
fig.line(wavelength, flux, line_color='black', legend_label='Spectrum')

# De-redshifted spectrum
fig.line(de_redshifted_wavelength, flux, line_color='red', legend_label='De-redshifted Spectrum')

# OIII
oiii_rest_wavelength = 5007
fig.add_layout(Span(location=oiii_rest_wavelength, dimension='height', line_color='red', line_dash='dashed', line_width=1))

# H-alpha
halpha_rest_wavelength = 6563
fig.add_layout(Span(location=halpha_rest_wavelength, dimension='height', line_color='green', line_dash='dashed', line_width=1))

# H-beta
hbeta_rest_wavelength = 4861
fig.add_layout(Span(location=hbeta_rest_wavelength, dimension='height', line_color='blue', line_dash='dashed', line_width=1))

show(fig)

### Step 5: Calculating the Radial Velocity

The radial velocity ($v$) can be calculated from the redshift using the formula:

$$ v = c \cdot z $$

where $c$ is the speed of light (approximately $3 \times 10^5 \, \text{km/s}$).

In [31]:
speed_of_light = 3e5  # Speed of light in km/s
radial_velocity = speed_of_light * redshift
print(f"Radial Velocity (v): {radial_velocity:.2f} km/s")

Radial Velocity (v): 13720.79 km/s


We find that this galaxy is moving away from us at a speed of $13720.79 \; \mathrm{km/s}$. We can use the radial velocity along with the distance to the galaxy to apply Hubble's law and calculate the Hubble constant.

## Using Your Own Galaxies

Now that you've seen how we can download and analyze galaxy spectra from SDSS, you can try it with your own galaxies. Return to the top of this notebook and input the coordinates of each of your galaxies. Re-run the cells and calculate the redshift and radial velocity. Be sure to record your results in the [Google Sheet](https://docs.google.com/spreadsheets/d/1mVRovTF8C1UJkQakQG_E9emnpJIRb1rdNx6UPNcm3rs/edit?usp=sharing).

In some cases, your galaxy's emission lines might not be one of the three that have been included here. If that's the case, let me know and we can figure out which line it really is.